In [2]:
import os

In [3]:
%pwd

'c:\\Users\\mrudu\\IKMLCourse\\IKProject\\MediWatch_project\\research'

In [8]:
#os.chdir("C:/Users/Dheer/OneDrive/Desktop/MLOps/MediWatch_project")
os.chdir("..")

In [9]:
%pwd

'c:\\Users\\mrudu\\IKMLCourse\\IKProject\\MediWatch_project'

In [17]:
# You can find the Password in Dagshub under Settings -> Tokens tab. Use the Default access token -> copy
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/gundalar/MediWatch_project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="gundalar"
os.environ["MLFLOW_TRACKING_PASSWORD"]="2a265003a41f12e4bc7098383654cecc57d8095c"

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [12]:
from Mediwatch_project.constants import *
from Mediwatch_project.utils.common import read_yaml, create_directories, save_json

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.model_params
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = Path(config.metric_file_name),
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/gundalar/MediWatch_project.mlflow",
        )
        return model_evaluation_config

In [14]:
from dataclasses import dataclass
from pathlib import Path
import json
import joblib
import numpy as np
import pandas as pd
from urllib.parse import urlparse

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
import mlflow

In [15]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        acc = accuracy_score(actual, pred)
        prec = precision_score(actual, pred, average="binary",zero_division=0)
        rec = recall_score(actual, pred, average="binary",zero_division=0)
        f1 = f1_score(actual, pred, average="binary",zero_division=0)
        return acc, prec, rec, f1    

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            acc, prec, rec, f1 = self.eval_metrics(test_y, predicted_qualities)
            
            scores = {
                "accuracy": float(acc),
                "precision": float(prec),
                "recall": float(rec),
                "f1_score": float(f1),
                "n_samples": int(len(test_y)),
                "positive_rate": float(np.mean(test_y == 1)),
            }

            out_path = Path(self.config.metric_file_name)
            out_path.parent.mkdir(parents=True, exist_ok=True)
            with open(out_path, "w") as f:
                json.dump(scores, f, indent=2)
            

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("accuracy", acc)
            mlflow.log_metric("precision", prec)
            mlflow.log_metric("recall", rec)
            mlflow.log_metric("f1_score", f1)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="XGB Classifier")
            else:
                mlflow.sklearn.log_model(model, "model")

In [16]:
try:
    config_mgr = ConfigurationManager()
    eval_cfg = config_mgr.get_model_evaluation_config()
    evaluator = ModelEvaluation(config=eval_cfg)
    evaluator.log_into_mlflow()
except Exception as e:
    raise

[2025-08-10 16:34:28,461: INFO: common: Raw YAML content from WindowsPath('config/config.yaml'): {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://raw.githubusercontent.com/dheeraj24797/MediWatch_project/main/diabetic_data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/diabetic_data.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/diabetic_data.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_path': 'artifacts/data_transformation/train.csv', 'test_data_path': 'artifacts/data_transformation/test.csv', 'model_name': 'model.joblib'}, 'model_evaluation': {'root_dir': 'artifacts/model_evaluation', 'test_data_path': 'artifac

c:\Users\mrudu\IKMLCourse\IKProject\MediWatch_project\env\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'XGB Classifier' already exists. Creating a new version of this model...
2025/08/10 16:34:39 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: XGB Classifier, version 5
Created version '5' of model 'XGB Classifier'.
